1. 라벨 인코딩할 때 원본/숫자 어떻게 변한건지 확인할 수 있도록

# 0. GPU check

* 이 코드는 Nvidia GPU를 사용하는 컴퓨터에서, train / test 데이터가 분리되어있는 csv 파일을 사용하는 것을 전제로 작성됨

In [1]:
import torch

if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    print("device_count: {}".format(device_count))
    for device_num in range(device_count):
        print("device {} capability {}".format(
            device_num,
            torch.cuda.get_device_capability(device_num)))
        print("device {} name {}".format(
            device_num, 
            torch.cuda.get_device_name(device_num)))
else:
    print("no cuda device")

device_count: 1
device 0 capability (8, 6)
device 0 name NVIDIA GeForce RTX 3080


In [2]:
if torch.cuda.is_available() :
    device = torch.device("cuda:0")
else : 
    device = torch.device("cpu")

In [3]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()
    
print_gpu_utilization()

GPU memory occupied: 436 MB.


* 모델 훈련과정에서 GPU 메모리 용량 초과 시, 개발서버 콘솔에서 직접 `nvidia-smi` 명령어 실행 후 메모리를 점유하고 있는 process의 PID를 찾아 `sudo kill -9 {pid}` 로 프로세스 종료해주면 됨

# 1. Import packages

In [4]:
## Need to check if packages are compatible
# !pip install accelerate nvidia-ml-py3
# !pip install datasets==2.4.0
# !pip install huggingface_hub==0.9.1
# !pip install transformers==4.22.1 # bf16, tf32 등 사용하려면 4.2 이상 필요
# !pip install pyarrow==9.0.0

* huggingface_hub와 transformers 간 호환가능한 버전 확인 필요
* 만약 성능 테스트를 위해 datasets api를 사용할거라면 datasets 역시 호환 가능 버전 확인해야 함
* 세 가지 dependencies를 사용한다는 가정 하에, pyarrow 라이브러리도 필요.

In [5]:
import transformers
import datasets
import huggingface_hub
import pyarrow

print(transformers.__version__)
print(datasets.__version__)
print(huggingface_hub.__version__)
print(pyarrow.__version__)

# 4.22.1
# 2.4.0
# 0.9.1
# 9.0.0

4.22.1
2.4.0
0.9.1
9.0.0


In [6]:
import os
import re
import math
import numpy as np
import pandas as pd

# 'You can use tf32' if you are acessing Ampere hardware
import torch
torch.backends.cuda.matmul.allow_tf32 = True

from datasets import load_dataset, load_metric, ClassLabel
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, precision_score, recall_score, f1_score

import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.examples.pbt_transformers.utils import (
    download_data,
    build_compute_metrics_fn,
)
from ray.tune.schedulers import PopulationBasedTraining
from transformers import (
    glue_tasks_num_labels,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    GlueDataset,
    GlueDataTrainingArguments,
    TrainingArguments,
    EarlyStoppingCallback
)

# 2. Import Data

* xxx_train.csv, xxx_test.csv 파일은 아래 형식으로 전처리된 csv 파일이어야 함 (column name: `text`, `label`)


<table class="features-table">
  <tr>
    <th class="mdc-text-light-green-600", style="text-align:center">
    text
    </th>
    <th class="mdc-text-purple-600", style="text-align:center">
    label
    </th>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      Ok lar... Joking wif u oni...
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      U dun say so early hor... U c already then say...
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      Nah I don't think he goes to usf, he lives around here though
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
</table>

In [7]:
data_name = "financial_news" ## covid_articles / financial_news / IMDB / naver_movie_review / spam

dataset = load_dataset('csv', data_files={'train': f'../data_splited/{data_name}_train.csv',
                                          'test': f'../data_splited/{data_name}_test.csv'})
dataset

Using custom data configuration default-b54327dcafa3f6de
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-b54327dcafa3f6de/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8602
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2151
    })
})

# 3. Data Preprocessing

* load_dataset 함수로 불러온 데이터를 수정할 때는 수정 내용을 담은 함수를 만들고, 이를 map 함수로 각 원소에 적용함 ([링크](https://huggingface.co/docs/datasets/v1.4.0/processing.html#processing-data-row-by-row)에서 확인)

In [8]:
## remove specal characters

def remove_sp(example):
    example["text"]=re.sub(r'[^a-z|A-Z|0-9|ㄱ-ㅎ|ㅏ-ㅣ|가-힣| ]+', '', str(example["text"]))
    return example

dataset = dataset.map(remove_sp)

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-b54327dcafa3f6de/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-0a289c34d582783d.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-b54327dcafa3f6de/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-4e0c7b4a17783a4a.arrow


In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8602
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2151
    })
})

In [10]:
## label encoding

labels = list(set(dataset["train"]["label"] + dataset["test"]["label"]))
num_labels = len(labels)

def encoding_label(example):
    str_to_int = ClassLabel(num_classes=num_labels, names=labels)
    example["label"]=str_to_int.str2int(example["label"])
    return example

if type(labels[0]) == str:
    dataset = dataset.map(encoding_label)
    
print(num_labels)

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-b54327dcafa3f6de/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-59064a61b9ca7b76.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-b54327dcafa3f6de/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-85fcd806e22e628a.arrow


3


In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8602
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2151
    })
})

In [12]:
# For IMDB and Naver Movie Review, 

# Make imbalanced data to test model performance (label 0:label 1 = 8:2)
# https://discuss.huggingface.co/t/huggingface-datasets-convert-a-dataset-to-pandas-and-then-convert-it-back/14708/3

# df_train = pd.DataFrame(dataset['train'])
# df_train_0 = df_train[df_train["label"]==0]
# df_train_1 = df_train[df_train["label"]==1].sample(frac=1)[0:math.floor(len(df_train[df_train['label']==0])*0.2)]
# dataset["train"] = datasets.Dataset.from_pandas(pd.concat([df_train_0,df_train_1]), preserve_index=False)
# dataset

# 4. Load PLM & Tokenizing

In [13]:
# model_name = "bert-base-cased"
# model_name = "klue/bert-base"

# model_name = "bert-base-multilingual-cased"

# model_name = "xlm-roberta-base"
# model_name = "klue/roberta-base"

In [14]:
# Download cache tokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [15]:
def tokenize_function(examples):
    tokenized_batch = tokenizer(examples["text"], padding="max_length", truncation=True) # padding : ['longest', 'max_length', 'do_not_pad']
    return tokenized_batch

In [16]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [17]:
# train_dataset = tokenized_datasets["train"].shuffle(seed=1919).select(range(0,math.floor(len(tokenized_datasets["train"])*0.7)))
# eval_dataset = tokenized_datasets["train"].shuffle(seed=1919).select(range(math.floor(len(tokenized_datasets["train"])*0.7), len(tokenized_datasets["train"])))
# test_dataset = tokenized_datasets["test"]

In [18]:
# data for test
train_dataset = tokenized_datasets["train"].shuffle(seed=1919).select(range(2000))
eval_dataset = tokenized_datasets["train"].shuffle(seed=1919).select(range(2000))
test_dataset = tokenized_datasets["test"]

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/csv/default-b54327dcafa3f6de/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-c163606819601bfe.arrow


# 5. Check class weights

In [19]:
def class_weight(train_dataset) :
    
    train_labels = np.array(train_dataset["label"])
    class_weights = compute_class_weight(class_weight = 'balanced', classes = np.unique(train_labels), y = train_labels)
    
    weights = torch.tensor(class_weights, dtype = torch.float)
    
    return weights

In [20]:
weights = class_weight(train_dataset)
print(weights)

tensor([1.1696, 0.8749, 0.9980])


# 6. Modeling

In [21]:
## Customize training strategy

task_data_dir = "test-model"
gpus_per_trial = 1
cpus_per_trial = 16
n_trials = 5
metric = load_metric("accuracy")
seed = 818

In [22]:
# Download model and features

config = AutoConfig.from_pretrained(
    model_name, 
    num_labels=num_labels
)

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        model_name,
        config=config
        )

In [23]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# https://stackoverflow.com/questions/69087044/early-stopping-in-bert-trainer-instances
# def compute_metrics(p):    
#     pred, labels = p
#     pred = np.argmax(pred, axis=1)
#     accuracy = accuracy_score(y_true=labels, y_pred=pred)
#     recall = recall_score(y_true=labels, y_pred=pred, average = 'weighted')
#     precision = precision_score(y_true=labels, y_pred=pred, average = 'weighted')
#     f1 = f1_score(y_true=labels, y_pred=pred, average = 'weighted')    
# return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

```python
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=1,   # batch size per device during training
    per_device_eval_batch_size=10,   # batch size for evaluation
    warmup_steps=1000,               # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=200,               # How often to print logs
    do_train=True,                   # Perform training
    do_eval=True,                    # Perform evaluation
    evaluation_strategy="epoch",     # evalute after each epoch
    gradient_accumulation_steps=64,  # total number of steps before back propagation
    fp16=True,                       # Use mixed precision
    fp16_opt_level="02",             # mixed precision mode
    run_name="ProBert-BFD-MS",       # experiment name
    seed=3                           # Seed for experiment reproducibility 3x3
)
```

In [24]:
training_args = TrainingArguments(
    output_dir=".",
    learning_rate=2e-5, # config
    do_train=True,
    do_eval=True,
    no_cuda=gpus_per_trial <= 0,
    evaluation_strategy="steps",
    save_strategy="steps",
    metric_for_best_model="accuracy",
    greater_is_better=True,
    load_best_model_at_end=True,
    num_train_epochs=2,  # config
    max_steps=-1,  # config
    per_device_train_batch_size=8,  # config
    per_device_eval_batch_size=8,  # config
    warmup_steps=0,
    warmup_ratio=0.1,  # config
    weight_decay=0.1,  # config
    logging_dir="./logs",
    skip_memory_metrics=True,
    report_to="none",
    fp16=True,
    # bf16=True,
    # tf32=True,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    seed=seed,
    eval_steps = 50
    )
    
# trainer = Trainer(
#     model_init=model_init,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=eval_dataset,
#     compute_metrics=compute_metrics,
#     )

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss
        weight = weights.to(device)
        loss_fct = torch.nn.CrossEntropyLoss(weight=weight)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss
    
trainer = CustomTrainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
    )

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/f6d161e8f5f6f2ed433fb4023d6cb34146506b3f/pytorch_model.bin
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassificat

In [25]:
# Hyperparameter tuning with ray tune

tune_config = {
#     "per_device_train_batch_size": tune.choice([2, 4, 8]),
    "num_train_epochs": tune.choice([2, 5]),
#     "num_train_epochs": [x for x in range(2, 21)],
}

# PopulationBasedTraining
# worker might copy the model parameters from a better performing worker or explore new hyperparameters by changing the current values randomly
# cf. ASHAScheduler
scheduler = PopulationBasedTraining(
    time_attr="training_iteration",
    metric="eval_accuracy",
    mode="max",
    perturbation_interval=1,
    hyperparam_mutations={
#         "num_train_epochs": [x for x in range(2, 21)],
        "weight_decay": tune.uniform(0.0, 0.3), # tune.uniform(1, 10) == np.random.uniform(1, 10)
        "learning_rate": tune.uniform(1e-5, 5e-5),
        "warmup_ratio": tune.uniform(0.0, 0.3),
#         # Perturb factor3 by changing it to an adjacent value, e.g.
#         # 10 -> 1 or 10 -> 100. Resampling will choose at random.
#         "factor_3": [1, 10, 100, 1000, 10000],
#         # Using tune.choice is NOT equivalent to the above.
#         # factor_4 is treated as a continuous hyperparameter.
#         "factor_4": tune.choice([1, 10, 100, 1000, 10000]),
    },
)


reporter = CLIReporter(
    parameter_columns={
        "weight_decay": "w_decay",
        "learning_rate": "lr",
        "per_device_train_batch_size": "train_bs/gpu",
        "num_train_epochs": "num_epochs",
    },
    metric_columns=["eval_accuracy", "eval_loss", "epoch", "training_iteration"],
)

result = trainer.hyperparameter_search(
    direction = "maximize",
    hp_space = lambda _: tune_config,
    backend="ray",
    n_trials=n_trials,
    resources_per_trial={"cpu": cpus_per_trial, "gpu": gpus_per_trial},
    scheduler=scheduler,
    keep_checkpoints_num=1,
    checkpoint_score_attr="training_iteration",
    stop=None,
    progress_reporter=reporter,
    local_dir="./test-results",
    name="tune_transformer_pbt",
    log_to_file=True,
)

2022-10-19 01:25:12,169	INFO worker.py:1518 -- Started a local Ray instance.
/opt/conda/lib/python3.8/site-packages/ray/tune/tune.py:505: UserWarning: Consider boosting PBT performance by enabling `reuse_actors` as well as implementing `reset_config` for Trainable.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/ray/tune/trainable/function_trainable.py:642: DeprecationWarning: `checkpoint_dir` in `func(config, checkpoint_dir)` is being deprecated. To save and load checkpoint in trainable functions, please use the `ray.air.session` API:

from ray.air import session

def train(config):
    # ...
    session.report({"metric": metric}, checkpoint=checkpoint)

For more information please see https://docs.ray.io/en/master/ray-air/key-concepts.html#session

  warnings.warn(
2022-10-19 01:25:19,474	WARNING trial_runner.py:1575 -- You are trying to access _search_alg interface of TrialRunner in TrialScheduler, which is being restricted. If you believe it is reasonable for your scheduler

== Status ==
Current time: 2022-10-19 01:25:19 (running for 00:00:00.17)
Memory usage on this node: 9.3/31.1 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 16.0/20 CPUs, 1.0/1 GPUs, 0.0/14.75 GiB heap, 0.0/7.37 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/BERT_classification_binary/test-results/tune_transformer_pbt
Number of trials: 5/5 (4 PENDING, 1 RUNNING)
+------------------------+----------+--------------------+-----------+-------------+----------------+--------------+
| Trial name             | status   | loc                |   w_decay |          lr | train_bs/gpu   |   num_epochs |
|------------------------+----------+--------------------+-----------+-------------+----------------+--------------|
| _objective_e4e31_00000 | RUNNING  | 172.17.0.3:3800968 | 0.182321  | 1.20392e-05 |                |            5 |
| _objective_e4e31_00001 | PENDING  |                    | 0.252205  | 4.56581e-05 |                |            5 

(_objective pid=3800968) Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.bias']
(_objective pid=3800968) - This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(_objective pid=3800968) - This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(_objective pid=3800968) Some weights of XLMRobertaForSequenceClassification were not ini

== Status ==
Current time: 2022-10-19 01:25:26 (running for 00:00:07.41)
Memory usage on this node: 13.9/31.1 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 16.0/20 CPUs, 1.0/1 GPUs, 0.0/14.75 GiB heap, 0.0/7.37 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/BERT_classification_binary/test-results/tune_transformer_pbt
Number of trials: 5/5 (4 PENDING, 1 RUNNING)
+------------------------+----------+--------------------+-----------+-------------+----------------+--------------+
| Trial name             | status   | loc                |   w_decay |          lr | train_bs/gpu   |   num_epochs |
|------------------------+----------+--------------------+-----------+-------------+----------------+--------------|
| _objective_e4e31_00000 | RUNNING  | 172.17.0.3:3800968 | 0.182321  | 1.20392e-05 |                |            5 |
| _objective_e4e31_00001 | PENDING  |                    | 0.252205  | 4.56581e-05 |                |            5

  2%|▏         | 7/310 [00:04<03:21,  1.50it/s]


...

Result for _objective_e4e31_00001:
  date: 2022-10-19_02-22-37
  done: false
  episodes_total: 0
  epoch: 4.83
  eval_accuracy: 0.955
  eval_loss: 0.1521279215812683
  eval_runtime: 10.104
  eval_samples_per_second: 197.942
  eval_steps_per_second: 24.743
  experiment_id: d51dc8ce8bf84df49c596df2b4e08e77
  hostname: 3481a8a2ae33
  iterations_since_restore: 6
  node_ip: 172.17.0.3
  objective: 0.955
  pid: 3818090
  time_since_restore: 268.0360436439514
  time_this_iter_s: 43.65184760093689
  time_total_s: 1218.5414962768555
  timestamp: 1666146157
  timesteps_since_restore: 0
  timesteps_total: 0
  training_iteration: 6
  trial_id: e4e31_00001
  warmup_time: 0.0033435821533203125
  
(_objective pid=3818090) {'eval_loss': 0.1521279215812683, 'eval_accuracy': 0.955, 'eval_runtime': 10.104, 'eval_samples_per_second': 197.942, 'eval_steps_per_second': 24.743, 'epoch': 4.83}


(_objective pid=3818090) 
                                                 
100%|██████████| 250/250 [00:10<00:00, 24.87it/s]
                                                 
 99%|█████████▉| 307/310 [04:27<00:03,  1.03s/it]


== Status ==
Current time: 2022-10-19 02:22:42 (running for 00:57:23.49)
Memory usage on this node: 13.9/31.1 GiB
PopulationBasedTraining: 14 checkpoints, 3 perturbs
Resources requested: 16.0/20 CPUs, 1.0/1 GPUs, 0.0/14.75 GiB heap, 0.0/7.37 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/BERT_classification_binary/test-results/tune_transformer_pbt
Number of trials: 5/5 (1 RUNNING, 4 TERMINATED)
+------------------------+------------+--------------------+------------+-------------+----------------+--------------+-----------------+-------------+---------+----------------------+
| Trial name             | status     | loc                |    w_decay |          lr | train_bs/gpu   |   num_epochs |   eval_accuracy |   eval_loss |   epoch |   training_iteration |
|------------------------+------------+--------------------+------------+-------------+----------------+--------------+-----------------+-------------+---------+----------------------|
| _objective_e4e31_0000

100%|██████████| 310/310 [04:29<00:00,  1.15it/s]
2022-10-19 02:22:44,791	INFO tune.py:758 -- Total run time: 3445.45 seconds (3445.21 seconds for the tuning loop).


Result for _objective_e4e31_00001:
  date: 2022-10-19_02-22-37
  done: true
  episodes_total: 0
  epoch: 4.83
  eval_accuracy: 0.955
  eval_loss: 0.1521279215812683
  eval_runtime: 10.104
  eval_samples_per_second: 197.942
  eval_steps_per_second: 24.743
  experiment_id: d51dc8ce8bf84df49c596df2b4e08e77
  experiment_tag: 1_num_train_epochs=5
  hostname: 3481a8a2ae33
  iterations_since_restore: 6
  node_ip: 172.17.0.3
  objective: 0.955
  pid: 3818090
  time_since_restore: 268.0360436439514
  time_this_iter_s: 43.65184760093689
  time_total_s: 1218.5414962768555
  timestamp: 1666146157
  timesteps_since_restore: 0
  timesteps_total: 0
  training_iteration: 6
  trial_id: e4e31_00001
  warmup_time: 0.0033435821533203125
  
== Status ==
Current time: 2022-10-19 02:22:44 (running for 00:57:25.22)
Memory usage on this node: 13.7/31.1 GiB
PopulationBasedTraining: 14 checkpoints, 3 perturbs
Resources requested: 0/20 CPUs, 0/1 GPUs, 0.0/14.75 GiB heap, 0.0/7.37 GiB objects (0.0/1.0 accelerator_

In [26]:
result

BestRun(run_id='e4e31_00001', objective=0.955, hyperparameters={'num_train_epochs': 5, 'weight_decay': 0.2522046274267129, 'learning_rate': 4.5658059231450835e-05, 'warmup_ratio': 0.2975867993967936})

In [27]:
for n, v in result.hyperparameters.items():
    setattr(trainer.args, n, v)

In [28]:
# trainer.args

In [29]:
trainer.train()

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/f6d161e8f5f6f2ed433fb4023d6cb34146506b3f/pytorch_model.bin
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassificat

Step,Training Loss,Validation Loss,Accuracy
50,No log,1.056868,0.423500
100,No log,0.606412,0.774500
150,No log,0.387631,0.859000
200,No log,0.315516,0.897000
250,No log,0.209575,0.931500
300,No log,0.152128,0.955000


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
/opt/conda/lib/python3.8/site-packages/transformers/trainer.py:1808: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  nn.utils.clip_grad_norm_(
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignor

TrainOutput(global_step=310, training_loss=0.6109579763104839, metrics={'train_runtime': 270.2355, 'train_samples_per_second': 37.005, 'train_steps_per_second': 1.147, 'total_flos': 2626924362596352.0, 'train_loss': 0.6109579763104839, 'epoch': 4.99})

In [30]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8


{'eval_loss': 0.14639249444007874,
 'eval_accuracy': 0.964,
 'eval_runtime': 10.1166,
 'eval_samples_per_second': 197.694,
 'eval_steps_per_second': 24.712,
 'epoch': 4.99}

In [31]:
pred = trainer.predict(test_dataset=test_dataset)

The following columns in the test set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2151
  Batch size = 8


PredictionOutput(predictions=array([[-2.955 ,  2.96  ,  0.2139],
       [-2.098 ,  2.91  , -0.678 ],
       [-1.948 ,  3.555 , -1.296 ],
       ...,
       [-1.896 , -0.865 ,  2.848 ],
       [-0.4297,  0.509 , -0.1447],
       [ 3.252 , -1.106 , -2.4   ]], dtype=float16), label_ids=array([1, 1, 1, ..., 1, 2, 0]), metrics={'test_loss': 0.5664333701133728, 'test_accuracy': 0.8238028823802882, 'test_runtime': 11.0103, 'test_samples_per_second': 195.363, 'test_steps_per_second': 24.432})

In [32]:
pred

PredictionOutput(predictions=array([[-2.955 ,  2.96  ,  0.2139],
       [-2.098 ,  2.91  , -0.678 ],
       [-1.948 ,  3.555 , -1.296 ],
       ...,
       [-1.896 , -0.865 ,  2.848 ],
       [-0.4297,  0.509 , -0.1447],
       [ 3.252 , -1.106 , -2.4   ]], dtype=float16), label_ids=array([1, 1, 1, ..., 1, 2, 0]), metrics={'test_loss': 0.5664333701133728, 'test_accuracy': 0.8238028823802882, 'test_runtime': 11.0103, 'test_samples_per_second': 195.363, 'test_steps_per_second': 24.432})

In [33]:
label_test = list(pred.label_ids)
pred_test = list(map(lambda x: x.index(max(x)), pred.predictions.tolist()))

In [34]:
print(confusion_matrix(label_test, pred_test))

[[534  30  70]
 [ 40 677  77]
 [ 63  99 561]]


In [35]:
accuracy = accuracy_score(label_test, pred_test)
f1 = f1_score(label_test, pred_test, average = 'weighted')
recall = recall_score(label_test, pred_test, average = 'weighted')
precision = precision_score(label_test, pred_test, average = 'weighted')

print(accuracy)
print(f1)
print(recall)
print(precision)

0.8238028823802882
0.823590227002483
0.8238028823802882
0.8234733961395012


In [36]:
# model_path = "test-model"
# trainer.model.save_pretrained(model_path)
# tokenizer.save_pretrained(model_path)

# Reference

https://bo-10000.tistory.com/154  
https://huggingface.co/blog/ray-tune  
https://docs.ray.io/en/latest/tune/examples/pbt_transformers.html  
https://wood-b.github.io/post/a-novices-guide-to-hyperparameter-optimization-at-scale/#schedulers-vs-search-algorithms  
https://docs.ray.io/en/latest/tune/api_docs/search_space.html  
https://docs.ray.io/en/latest/tune/tutorials/tune-advanced-tutorial.html  
https://docs.ray.io/en/latest/tune/api_docs/schedulers.html  
https://blog.ml.cmu.edu/2018/12/12/massively-parallel-hyperparameter-optimization/  
https://docs.ray.io/en/latest/tune/faq.html  
https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#population-based-training-tune-schedulers-populationbasedtraining  
https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.hyperparameter_search  
https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#optuna-tune-search-optuna-optunasearch  
https://kyunghyunlim.github.io/nlp/ml_ai/2021/09/22/hugging_face_5.html  
